In [1]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.colab.patches import cv2_imshow
import numpy as np
from scipy import ndimage
from skimage.color import rgb2hsv
import os
from google.colab import drive
drive.mount('/content/drive')

low_apple_red = (150.0, 10.0, 15.0)      # mini setup (Hue, Saturation, Value)
high_apple_red = (180.0, 255.0, 255.0)   # max setup (Hue, Saturation, Value)
low_apple_raw = (0.0, 10.0, 15.0)        # mini setup (Hue, Saturation, Value)
high_apple_raw = (15.0, 255.0, 255.0)    # max setup (Hue, Saturation, Value)

# Input and output directories
input_folder = '/content/drive/MyDrive/data(1)/test/images/'  # Input folder
output_folder = '/content/drive/MyDrive/output_images/'       # Output folder

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# List all images in the input folder
image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Loop through each image
for image_file in image_files:
    # Read the image
    img_path = os.path.join(input_folder, image_file)
    img1 = cv2.imread(img_path)
    origin = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

    image_hsv = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)

    mask_red = cv2.inRange(image_hsv,low_apple_red, high_apple_red)
    mask_raw = cv2.inRange(image_hsv,low_apple_raw, high_apple_raw)
    mask = mask_red + mask_raw

    red_regions = cv2.bitwise_and(img1, img1, mask=mask)

    gray_red_regions = cv2.cvtColor(red_regions, cv2.COLOR_BGR2GRAY)

    equal = cv2.equalizeHist(gray_red_regions)

    denoised_img = cv2.GaussianBlur(equal, (5, 5), 1)     # (5,5)Smooths the image, reducing small, random noise while preserving edge information. (1)smoothing without excessive blur.

    kernel = np.ones((2, 2), np.uint8)
    edges = cv2.Canny(denoised_img, 50, 255)      # Lower Threshold (50): Suppresses weak edges (noise)    # Upper Threshold (255): Retains strong edges.
    edges = cv2.dilate(edges,kernel,iterations = 1)     # Expands the edges to close small gaps, making them more robust for contour detection.
    edges_inverted = cv2.bitwise_not(edges)     # Inverts the edges, making them white on a black background

    _, equal_binary = cv2.threshold(equal, 1, 255, cv2.THRESH_BINARY)     # Keeps all non-zero pixels as white. This works well for detecting objects against a black background.

    edges_equal = cv2.bitwise_and(equal_binary, edges_inverted)     # Combines the binary image and the inverted edges, retaining only regions that are both bright and edge-like.

    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(edges_equal, cv2.MORPH_OPEN, kernel, iterations = 2)     # Kernel Size (3,3): Slightly larger to clean up small irregularities.      # Iterations (2): Repeatedly applies the operation for better noise removal.

    cnts,_ = cv2.findContours(opening, cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)


    unique_centers = []

    # Threshold distance to consider centers as the same
    threshold_distance = 20

    c_num = 0
    for i, c in enumerate(cnts):
        # Find the minimum enclosing circle for the contour
        ((x, y), r) = cv2.minEnclosingCircle(c)

        # Only consider objects within the desired radius range
        if 4 <= r <= 40:
            # Check if the center is far enough from all existing unique centers
            is_new_center = all(
                np.linalg.norm(np.array((x, y)) - np.array(center)) > threshold_distance
                for center in unique_centers
            )

            if is_new_center:
                unique_centers.append((x, y))  # Add this center to the list
                c_num += 1  # Increment count

                # Draw circles based on size
                if r <= 15:
                    cv2.circle(origin, (int(x), int(y)), int(15), (0, 255, 0), 2)
                else:
                    cv2.circle(origin, (int(x), int(y)), int(r), (0, 255, 0), 2)

    # Add a label to the image
    cv2.putText(origin, f"apples: {c_num}", (20, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 2)  # Red text

    result_bgr = cv2.cvtColor(origin, cv2.COLOR_RGB2BGR)
    output_path = os.path.join(output_folder, f"processed_{image_file}")
    cv2.imwrite(output_path, result_bgr)

print(f"Processing completed. Processed images saved to: {output_folder}")

Mounted at /content/drive
Processing completed. Processed images saved to: /content/drive/MyDrive/output_images/
